In [18]:
import numpy as np
import matplotlib
import cvxpy
import conrad

from matplotlib import pyplot as plt
from conrad import Case

In [19]:
# Define dimensions of problem
m_target = 100
m_oar = 400
m = m_target + m_oar
n = 200

# Structure labels
lab_tum = 0
lab_oar = 1

# Generate random beam matrix
A_target = np.random.rand(m_target, n)
A_oar = 0.5 * np.random.rand(m_oar, n)
A = np.vstack((A_target, A_oar))

label_order = [lab_tum, lab_oar]
voxel_labels = [lab_tum] * m_target + [lab_oar] * m_oar

In [20]:
# Prescription for each structure
rx = [{'label': lab_tum, 'name': 'tumor', 'is_target': True,  'dose': 1., 'constraints': None},
      {'label': lab_oar,  'name': 'oar',   'is_target': False, 'dose': 0., 'constraints': None}]

In [23]:
# Construct and solve unconstrained case
cs = Case(A, voxel_labels, label_order, rx)
cs.plan("ECOS", "dvh_no_slack", verbose = 1)
cs.plot(show = True)

('running solver...',)

ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +5.249e+01  +5.249e+01  +5e+02  6e-01  6e-02  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  +5.025e+01  +5.195e+01  +1e+02  8e-02  7e-03  2e+00  3e-01  0.9890  3e-01   1  2  2 |  0  0
 2  +5.199e+01  +5.205e+01  +2e+01  7e-03  7e-04  7e-02  4e-02  0.8861  1e-02   1  2  2 |  0  0
 3  +5.247e+01  +5.249e+01  +2e+00  6e-04  6e-05  2e-02  6e-03  0.9088  7e-02   1  1  1 |  0  0
 4  +5.250e+01  +5.250e+01  +2e-02  6e-06  6e-07  2e-04  7e-05  0.9889  1e-04   1  1  1 |  0  0

OPTIMAL (within feastol=6.5e-06, reltol=4.4e-04, abstol=2.3e-02).
Runtime: 0.089145 seconds.

('status: optimal',)
('optimal value: -0.0003103909145',)


In [22]:
# Add DVH constraints and resolve case
cs.add_dvh_constraint(lab_tum, 1.05, 30., '<')
cs.add_dvh_constraint(lab_tum, 0.8, 20., '>')
cs.add_dvh_constraint(lab_oar, 0.5, 50, '<')

# Solve without slacks using a single pass
cs.plan("ECOS", "dvh_no_slack", plot = True)

# Solve without slacks using 2 passes
cs.plan("ECOS", "dvh_no_pass", "dvh_2pass", plot = True)

('running solver...',)

ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +5.136e+01  -2.074e+02  +2e+03  7e-01  4e+00  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  +4.896e+01  +8.089e+00  +8e+02  9e-02  7e-01  3e+00  5e-01  0.9249  3e-01   1  0  0 |  0  0
 2  +4.938e+01  +2.208e+01  +8e+02  6e-02  4e-01  1e+00  5e-01  0.2660  8e-01   0  0  1 |  0  0
 3  +5.205e+01  +4.936e+01  +9e+01  6e-03  4e-02  8e-02  6e-02  0.8836  5e-03   1  0  0 |  0  0
 4  +5.244e+01  +5.208e+01  +2e+01  7e-04  5e-03  4e-03  1e-02  0.8699  6e-02   1  1  1 |  0  0
 5  +5.250e+01  +5.249e+01  +3e-01  1e-05  6e-05  5e-05  2e-04  0.9865  1e-03   1  1  1 |  0  0
 6  +5.250e+01  +5.250e+01  +3e-03  1e-07  7e-07  5e-07  2e-06  0.9890  1e-04   1  1  1 |  0  0

OPTIMAL (within feastol=7.2e-07, reltol=5.3e-05, abstol=2.8e-03).
Runtime: 0.289365 seconds.

('status: optimal',)
('o

ValueError: zero-size array to reduction operation maximum which has no identity

In [6]:
# Additional DVH constraint makes no-slack problem infeasible
cs.add_dvh_constraint(lab_oar, 0.55, 10, '>')

# Solving without slacks will result in infeasibility
# cs.plan("ECOS")

# Solve with slacks using a single pass
cs.plan("ECOS", plot = True)
cs.plan("ECOS", "dvh_2pass", plot = True)

('running solver...',)

ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +5.239e+01  +4.613e+01  +4e+03  7e-01  2e+00  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  +5.076e+01  +4.907e+01  +3e+03  3e-01  9e-01  2e+00  1e+00  0.3406  3e-01   0  0  0 |  0  0
 2  +5.004e+01  +5.077e+01  +2e+03  1e-01  4e-01  2e+00  7e-01  0.7271  5e-01   0  0  0 |  0  0
 3  +5.106e+01  +5.145e+01  +1e+03  6e-02  1e-01  1e+00  4e-01  0.6553  3e-01   0  0  0 |  0  0
 4  +5.206e+01  +5.204e+01  +1e+02  5e-03  1e-02  4e-02  5e-02  0.9334  5e-02   0  0  0 |  0  0
 5  +5.248e+01  +5.248e+01  +2e+01  4e-04  1e-03  9e-03  7e-03  0.9008  7e-02   1  1  1 |  0  0
 6  +5.250e+01  +5.250e+01  +3e+00  7e-05  2e-04  1e-03  1e-03  0.8113  7e-03   1  1  1 |  0  0
 7  +5.250e+01  +5.250e+01  +3e+00  7e-05  2e-04  8e-04  1e-03  0.4151  7e-01   1  1  1 |  0  0
 8  +5.250e+01  +5.250e

ValueError: zero-size array to reduction operation maximum which has no identity